# Creation of the final video

In [13]:
from pathlib import Path
import subprocess
import os

In [14]:
VIDEO_NUMBER = "7"
PROJECT_ROOT = Path().resolve().parent.parent
INPUT_VIDEO_PATH = str(PROJECT_ROOT / "data" / f"recording_{VIDEO_NUMBER}" / f"Tracked_output_{VIDEO_NUMBER}.mp4")
INPUT_VIDEO_LANE = str(PROJECT_ROOT / "data" / f"recording_{VIDEO_NUMBER}" / f"Reconstructed_trajectory_deformed_{VIDEO_NUMBER}.mp4")
INPUT_VIDEO_SPHERE = str(PROJECT_ROOT / "data" / f"recording_{VIDEO_NUMBER}" / f"Rotating_sphere.mp4")

OUTPUT_PATH = str(PROJECT_ROOT / "data" / f"recording_{VIDEO_NUMBER}" / f"Final_{VIDEO_NUMBER}.mp4")

In [15]:
if os.path.exists(OUTPUT_PATH):
        os.remove(OUTPUT_PATH)

In [16]:
# Step 1: Get the height of the first video
def get_video_height(video_path):
    # Use ffprobe to get video dimensions
    command = [
        'ffprobe', 
        '-v', 'error', 
        '-select_streams', 'v:0', 
        '-show_entries', 'stream=height', 
        '-of', 'default=noprint_wrappers=1:nokey=1', 
        video_path
    ]
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    height = int(result.stdout.decode().strip())
    return height

# Get the height of the first video (left video)
left_video_height = get_video_height(INPUT_VIDEO_PATH)

# Step 2: Resize the second video (right video) to match the height of the first video
# Construct the ffmpeg command for resizing the second video
resized_right_video = "resized_right_video.mp4"
padded_right_video = "padded_right_video.mp4"
resize_command = [
    'ffmpeg', 
    '-i', INPUT_VIDEO_LANE, 
    '-vf', f"scale=-1:{left_video_height}",  # Preserve aspect ratio and set height
    '-c:v', 'libx264', 
    '-preset', 'fast', 
    '-crf', '18', 
    resized_right_video
]
subprocess.run(resize_command, check=True)

# Add a 5-pixel black border to the left
pad_command = [
    "ffmpeg",
    "-i", str(resized_right_video),
    "-vf", "pad=width=iw+5:height=ih:x=5:y=0:color=black",
    "-c:v", "libx264",
    "-preset", "fast",
    "-crf", "18",
    str(padded_right_video)
]
subprocess.run(pad_command, check=True)

# Step 3: Resize the sphere video to a smaller size (you can adjust the size here)
resized_sphere_video = "resized_sphere_video.mp4"
padded_sphere_video = "padded_sphere_video.mp4"
sphere_width = 300  # Set the desired width for the sphere video
resize_sphere_command = [
    'ffmpeg', 
    '-i', INPUT_VIDEO_SPHERE, 
    '-vf', f"scale={sphere_width}:-1",  # Adjust width, keep aspect ratio
    '-c:v', 'libx264', 
    '-preset', 'fast', 
    '-crf', '18', 
    resized_sphere_video
]
subprocess.run(resize_sphere_command, check=True)

# Add a 5-pixel black border around the entire video
pad_command = [
    "ffmpeg",
    "-i", str(resized_sphere_video),
    "-vf", "pad=width=iw+5:height=ih+5:x=5:y=0:color=black",
    "-c:v", "libx264",
    "-preset", "fast",
    "-crf", "18",
    str(padded_sphere_video)
]
subprocess.run(pad_command, check=True)

# Step 4: Overlay the sphere video on the top-right corner of the left video
# We will use ffmpeg's overlay filter to place the resized sphere video on top of the left video
overlayed_left_video = "overlayed_left_video.mp4"
overlay_command = [
    'ffmpeg', 
    '-i', INPUT_VIDEO_PATH, 
    '-i', padded_sphere_video, 
    '-filter_complex', f"[0:v][1:v]overlay=W-w-0:0",  # Top-right corner, with 10px margin
    '-c:v', 'libx264', 
    '-preset', 'fast', 
    '-crf', '18', 
    overlayed_left_video
]
subprocess.run(overlay_command, check=True)

# Step 5: Combine the overlayed left video and the resized right video side by side
combine_command = [
    'ffmpeg', 
    '-i', overlayed_left_video, 
    '-i', padded_right_video, 
    '-filter_complex', 
    f"[0:v][1:v]hstack=inputs=2",  # Concatenate the videos side by side
    '-c:v', 'libx264', 
    '-preset', 'fast', 
    '-crf', '18', 
    OUTPUT_PATH
]
subprocess.run(combine_command, check=True)

# Clean up temporary files (optional)
# Elimina i file se esistono
for file_path in [resized_right_video, resized_sphere_video, overlayed_left_video, padded_sphere_video, padded_right_video]:
    if os.path.exists(file_path):
        os.remove(file_path)

ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena